# Training Probes using Contrast-Consistent Search (CCS)

This notebook demonstrates how to train probes using the CCS approach from Burns et al. (2023). CCS is a method for learning probes that can extract beliefs from language models without using labeled data.

The key idea is to train a probe that:
1. Makes informative predictions (assigns high probabilities to at least one of the contrast pairs)
2. Makes consistent predictions (assigns complementary probabilities to contrast pairs)

For example, if we have a statement "The sky is blue", we create two contrast pairs:
- "The sky is blue" (positive)
- "The sky is not blue" (negative)

The probe should assign high probability to one of these and low probability to the other.

In [62]:
%load_ext autoreload
%autoreload 2

import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

import torch.nn as nn
import torch.nn.functional as F
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
class LoadModel:
    def __init__(self, model_name: str, layer_idx: int, token_pos: int, device: str = "cuda" if torch.cuda.is_available() else "cpu"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True).to(device)
        self.model.eval()
        self.layer_idx = layer_idx
        self.token_pos = token_pos
        self.device = device

    def extract_activation(self, text: str):
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        hidden_states = outputs.hidden_states  # tuple: (embeddings, layer1, layer2, ..., final layer)

        if self.layer_idx >= len(hidden_states):
            raise ValueError(f"Layer index {self.layer_idx} is out of range.")
        
        activation = hidden_states[self.layer_idx]  # (batch_size, seq_len, hidden_dim)
        
        # Often we take the activation at the final token
        activation = activation[:, self.token_pos, :]  # shape: (batch_size, hidden_dim)

        return activation.squeeze(0).cpu()  # Return a 1D numpy array
    

In [74]:
class MLPProbe(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.linear1 = nn.Linear(d, 100)
        self.linear2 = nn.Linear(100, 1)

    def forward(self, x):
        h = F.relu(self.linear1(x))
        o = self.linear2(h)
        return torch.sigmoid(o)

class CCS(object):
    def __init__(self, x0, x1, nepochs=1000, ntries=10, lr=1e-3, batch_size=-1, 
                 verbose=False, device="cpu", linear=True, weight_decay=0.01, var_normalize=False):
        # data
        self.var_normalize = var_normalize
        self.x0 = self.normalize(x0)
        self.x1 = self.normalize(x1)
        self.d = self.x0.shape[-1]

        # training
        self.nepochs = nepochs
        self.ntries = ntries
        self.lr = lr
        self.verbose = verbose
        self.device = device
        self.batch_size = batch_size
        self.weight_decay = weight_decay
        
        # probe
        self.linear = linear
        self.initialize_probe()
        self.best_probe = copy.deepcopy(self.probe)

        
    def initialize_probe(self):
        if self.linear:
            self.probe = nn.Sequential(nn.Linear(self.d, 1), nn.Sigmoid())
        else:
            self.probe = MLPProbe(self.d)
        self.probe.to(self.device)    


    def normalize(self, x):
        """
        Mean-normalizes the data x (of shape (n, d))
        If self.var_normalize, also divides by the standard deviation
        """
        normalized_x = x - x.mean(axis=0, keepdims=True)
        if self.var_normalize:
            normalized_x /= normalized_x.std(axis=0, keepdims=True)

        return normalized_x

        
    def get_tensor_data(self):
        """
        Returns x0, x1 as appropriate tensors (rather than np arrays)
        """
        x0 = torch.tensor(self.x0, dtype=torch.float, requires_grad=False, device=self.device)
        x1 = torch.tensor(self.x1, dtype=torch.float, requires_grad=False, device=self.device)
        return x0, x1
    

    def get_loss(self, p0, p1):
        """
        Returns the CCS loss for two probabilities each of shape (n,1) or (n,)
        """
        informative_loss = (torch.min(p0, p1)**2).mean(0)
        consistent_loss = ((p0 - (1-p1))**2).mean(0)
        return informative_loss + consistent_loss


    def get_acc(self, x0_test, x1_test, y_test):
        """
        Computes accuracy for the current parameters on the given test inputs
        """
        x0 = torch.tensor(self.normalize(x0_test), dtype=torch.float, requires_grad=False, device=self.device)
        x1 = torch.tensor(self.normalize(x1_test), dtype=torch.float, requires_grad=False, device=self.device)
        with torch.no_grad():
            p0, p1 = self.best_probe(x0), self.best_probe(x1)
        avg_confidence = 0.5*(p0 + (1-p1))
        predictions = (avg_confidence.detach().cpu().numpy() < 0.5).astype(int)[:, 0]
        acc = (predictions == y_test).mean()
        acc = max(acc, 1 - acc)

        return acc
    
        
    def train(self):
        """
        Does a single training run of nepochs epochs
        """
        x0, x1 = self.get_tensor_data()
        permutation = torch.randperm(len(x0))
        x0, x1 = x0[permutation], x1[permutation]
        
        # set up optimizer
        optimizer = torch.optim.AdamW(self.probe.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        
        batch_size = len(x0) if self.batch_size == -1 else self.batch_size
        nbatches = len(x0) // batch_size

        # Start training (full batch)
        for epoch in range(self.nepochs):
            for j in range(nbatches):
                x0_batch = x0[j*batch_size:(j+1)*batch_size]
                x1_batch = x1[j*batch_size:(j+1)*batch_size]
            
                # probe
                p0, p1 = self.probe(x0_batch), self.probe(x1_batch)

                # get the corresponding loss
                loss = self.get_loss(p0, p1)

                # update the parameters
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        return loss.detach().cpu().item()
    
    def repeated_train(self):
        best_loss = np.inf
        for train_num in range(self.ntries):
            self.initialize_probe()
            loss = self.train()
            if loss < best_loss:
                self.best_probe = copy.deepcopy(self.probe)
                best_loss = loss

        return best_loss

### Example Usage

In [75]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # Replace with your model
layer_idx = -1  # Example layer
token_pos = -1

trainer = LoadModel(model_name, layer_idx, token_pos)

/Users/ruijiacao/Documents/Ruijia/Research/eliciting_beliefs_llm/eliciting_beliefs_ai/.conda/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


### Example Usage

In [76]:
positive_texts = ["Captial of France is Paris"]
negative_texts = ["Captial of France is not Paris"]
positive_tests = [
    "Captial of China is not Beijing"
    "Captial of USA is Washington D.C.."
]

negative_tests = [
    "Captial of China is Beijing",
    "Captial of USA is not Washington D.C.."
]
pos_hs_train = torch.stack([trainer.extract_activation(text) for text in positive_texts]) 
neg_hs_train = torch.stack([trainer.extract_activation(text) for text in negative_texts])

pos_hs_test = torch.stack([trainer.extract_activation(text) for text in positive_tests]) 
neg_hs_test = torch.stack([trainer.extract_activation(text) for text in negative_tests])

In [78]:
# Train CCS without any labels
ccs = CCS(neg_hs_train, pos_hs_train)
ccs.repeated_train()
probe = ccs.best_probe
print(probe.type)
y_test = [0, 1]

# Evaluate
ccs_acc = ccs.get_acc(neg_hs_test, pos_hs_test, y_test)
print("CCS accuracy: {}".format(ccs_acc))

/var/folders/k0/p9bdwg2d103dmfr62rhs5sn80000gn/T/ipykernel_24413/3652077333.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x0 = torch.tensor(self.x0, dtype=torch.float, requires_grad=False, device=self.device)
/var/folders/k0/p9bdwg2d103dmfr62rhs5sn80000gn/T/ipykernel_24413/3652077333.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x1 = torch.tensor(self.x1, dtype=torch.float, requires_grad=False, device=self.device)


<bound method Module.type of Sequential(
  (0): Linear(in_features=2048, out_features=1, bias=True)
  (1): Sigmoid()
)>
CCS accuracy: 1.0


/var/folders/k0/p9bdwg2d103dmfr62rhs5sn80000gn/T/ipykernel_24413/3652077333.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x0 = torch.tensor(self.normalize(x0_test), dtype=torch.float, requires_grad=False, device=self.device)
/var/folders/k0/p9bdwg2d103dmfr62rhs5sn80000gn/T/ipykernel_24413/3652077333.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x1 = torch.tensor(self.normalize(x1_test), dtype=torch.float, requires_grad=False, device=self.device)


### Testing